In [9]:
# Librerias
import os
from bs4 import BeautifulSoup
from unidecode import unidecode
import datetime
import traceback
import csv
import pandas as pd
import jellyfish
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [10]:
# Parámetros
inputFolder = "1-input"
processFolder = "2-process"
outputFolder = "3-output"
otherOutputFolder = "../4-modelling/1-input/"
otherOutputFolder2 = "../5-evaluation/1-input/main/"
logsFolder = "4-logs"

inputMainFolder = inputFolder + r"\main"
inputMastersFolder = inputFolder + r"\masters"

processMainFolder = processFolder + r"\main"
processSupportFolder = processFolder + r"\support"

dataVisualizationTopLimit = 20

In [11]:
# Funciones utilitarias
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def find_nth_right(haystack, needle, n):
    start = haystack.rfind(needle)
    while start >= 0 and n > 1:
        start = haystack.rfind(needle, 0, start-len(needle))
        n -= 1
    return start

def parseLineBreaksAndAccents(text):
  return unidecode(" ".join(text.split()))

def parseNames(text):
  return text.strip().title()

def findTags(tag, color):
  return tag.find("span", {"style": 'font-size:10.0pt;font-family:"Arial",sans-serif;mso-fareast-font-family:\n"Times New Roman";color:' + color })

def getChildIndex(mainChildTags, title, color):
  return next((index for index, tag in enumerate(mainChildTags) if ( parseNames(parseLineBreaksAndAccents(findTags(tag, color).text)) == title if findTags(tag, color) else False )), None)

def getSectionsIndexes(mainChildTags, color):
  sectionsIndexes = []
  sectionsTitle = ["Objetivo Laboral", "Experiencia Laboral", "Educacion", "Informatica", "Idiomas", "Otros Conocimientos"]
  
  for sectionTitle in sectionsTitle:
    sectionIndex = getChildIndex(mainChildTags, sectionTitle, color)
    sectionsIndexes.append(sectionIndex)
  
  sectionsIndexes.append(len(mainChildTags)-1)
  return sectionsIndexes

def getNextSectionIndexValid(sectionsIndexes, i):
  while(not sectionsIndexes[i]):
    i = i + 1

  return sectionsIndexes[i]

def getStartAndEndIndex(sectionsIndexes, i):
  return sectionsIndexes[i], getNextSectionIndexValid(sectionsIndexes, i+1)

def getCompare(data, fields):
  return (list(set([" ".join(x for x in [elem[y] for y in fields] if x) for elem in data if elem])))

def showNullValues(df):
  percentNulls = df.isnull().sum() / len(df)
  resultDf = pd.DataFrame({'columnName': df.columns, 'columnType': df.dtypes.to_dict().values(), 'percentNulls': percentNulls })
  resultDf = resultDf.sort_values('percentNulls', ascending=False).reset_index(drop=True)
  #Ajustes para la visualización
  resultDf.index = range(1,len(resultDf)+1)
  resultDf.loc[resultDf["columnType"] == "object", "columnType"] = "Categórico"
  resultDf.loc[resultDf["columnType"] == "float64", "columnType"] = "Numérico"
  resultDf.loc[resultDf["columnType"] == "int64", "columnType"] = "Numérico"

  resultDf["percentNulls"] =  (resultDf["percentNulls"]*100).map('{:,.2f}%'.format)

  display(resultDf)

def showAtypicalValues(df):
  originalDf = df.copy()
  columns = [columnName for columnName, columnType in df.dtypes.to_dict().items()]
  numericalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "contratado" ] and columnType == "float64" ]

  for column in columns:
    if column in numericalColumns:
      firstQuartil, thirdQuartil = np.percentile(df[column], 25), np.percentile(df[column], 75)
      interQuartilRange = thirdQuartil - firstQuartil
      bottomLimit, topLimit = firstQuartil - 3*interQuartilRange, thirdQuartil + 3*interQuartilRange

      # Reemplazando los valores
      df[column] = df[column].map(lambda x: True if (x < bottomLimit or x > topLimit) else False)
    else:
      df[column] = df[column].map(lambda x: False)
  
  percentAtypical = df.sum() / len(df)

  resultDf = pd.DataFrame({'columnName': df.columns, 'columnType': originalDf.dtypes.to_dict().values(), 'percentAtypical': percentAtypical })
  resultDf = resultDf.sort_values('percentAtypical', ascending=False).reset_index(drop=True)
  #Ajustes para la visualización
  resultDf.index = range(1,len(resultDf)+1)
  resultDf.loc[resultDf["columnType"] == "object", "columnType"] = "Categórico"
  resultDf.loc[resultDf["columnType"] == "float64", "columnType"] = "Numérico"
  resultDf.loc[resultDf["columnType"] == "int64", "columnType"] = "Numérico"

  resultDf["percentAtypical"] =  (resultDf["percentAtypical"]*100).map('{:,.2f}%'.format)

  display(resultDf)

In [12]:
# Funciones utilitarias de archivos
def readCsvAsDict(filePath, delimiter=",", encoding="utf-8", header=0, dtype={}):
  df = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header, dtype=dtype)
  df = df.replace(np.nan, '', regex=True)
  data = df.to_dict('records')
  return data

def readCsvAsDf(filePath, delimiter=",", encoding="utf-8", header=0, dtype={}):
  df = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header, dtype=dtype)
  df = df.replace(np.nan, '', regex=True)
  return df

def writeDictToCsv(data, pathCsv, encoding='utf-8'):
  with open(pathCsv, 'w', newline='', encoding=encoding) as f:
    if data:
      writer = csv.DictWriter(f, fieldnames=data[0].keys(), lineterminator='\n')
      writer.writeheader()
      writer.writerows(data)
    else:
      f.write("")

def writeDfToCsv(data, pathCsv, encoding='utf-8'):
  data.to_csv(path_or_buf = pathCsv, encoding = encoding, index=False)

In [13]:
def readAndWritePreprocessedData(df, masters):
  ## Decisiones propias para el modelo

  # Acotando registros por fecha de inicio
  df['fechaPostulacion']= pd.to_datetime(df['fechaPostulacion'])
  df = df[df["fechaPostulacion"] > datetime.datetime(2019,6,21,0,0,0)]

  # Eliminando columnas que son datos sensibles de los candidatos
  sensitiveColumns = [
    "nombreCompleto", "numeroDocumento", "fechaNacimiento", "direccion", "numeroCasa", "numeroCelular", "correoElectronico"
  ]
  df = df.drop(columns=sensitiveColumns)

  # Eliminando columnas que no se usarán, porque no son relevantes para el modelo
  nonRelevanceColumns = [
    "fechaPostulacion", "idConvocatoria", "objetivoLaboral", "descripcionUltimoTrabajo"
  ]
  df = df.drop(columns=nonRelevanceColumns)

  # Eliminando columnas normadas por Ley N° 26772: Igualdad de oportunidades y de trato
  lawRestrictiveColumns = [
    "estadoCivil", "paisNacimiento"
  ]
  df = df.drop(columns=lawRestrictiveColumns)

  # Aplicar las equivalencias a los campos: nombrePerfilConvocatoria, empresaUltimoTrabajo, nombreUltimoTrabajo, institucionUltimoEstudio, nombreUltimoEstudio
  equivalences = ["nombrePerfilConvocatoria", "empresaUltimoTrabajo", "nombreUltimoTrabajo", "institucionUltimoEstudio", "nombreUltimoEstudio"]

  # Mejorar las equivalencias poco a poco
  for index, equivalence in enumerate(equivalences):
    df = pd.merge(df, masters[index], on=equivalence)
    df[equivalence] = df["equivalencia"]
    df = df.drop(columns=["equivalencia", "cantidad", "igual"])

  ## Decisiones por la metodología

  # Analisis de valores nulos
  # Reemplazando las cadenas vacias a NaN
  categoricalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "contratado" ] and columnType == "object" ]
  for column in categoricalColumns:
    df[column] = df[column].replace('',None,regex = True).astype("object")

  # Reemplazando los ceros a NaN
  numericalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "contratado" ] and columnType == "float64" ]
  for column in numericalColumns:
    df[column] = df[column].replace(0,None).astype('float64')

  # Obteniendo el porcentaje de nulos por columna
  showNullValues(df)
  
  # Tecnica 1: Eliminar la columna si sobrepasa el 30% de nulos
  # Al ninguno superar el 30% de nulos, no se eliminará ninguna columna

  # Tecnica 2: Reemplazar NaN por aleatorizacion de valores no nulos, para variables categoricas
  np.random.seed(0)

  for column in categoricalColumns:
    df[column] = df[column].map(lambda x: x if not pd.isna(x) else np.random.choice(df[column].dropna().tolist()))

  showNullValues(df)

  # Tecnica 3: Reemplazar NaN por el promedio de valores no nulos, para variables numericas
  for column in numericalColumns:
    df[column] = df[column].map(lambda x: x if not pd.isna(x) else round(float(df[column].dropna().mean()), 1) )

  # Obteniendo el nuevo porcentaje de nulos por columna
  showNullValues(df)

  # Reemplazo de valores atipicos (outliers) (variables numericas)
  atypicalDf = df.copy()
  showAtypicalValues(atypicalDf)

  # Analizando valores atipicos extremos
  for column in numericalColumns:
    firstQuartil, thirdQuartil = np.percentile(df[column], 25), np.percentile(df[column], 75)
    interQuartilRange = thirdQuartil - firstQuartil
    bottomLimit, topLimit = firstQuartil - 3*interQuartilRange, thirdQuartil + 3*interQuartilRange

    # Reemplazando los valores
    df[column] = df[column].map(lambda x: bottomLimit if x < bottomLimit else x)
    df[column] = df[column].map(lambda x: topLimit if x > topLimit else x)

  atypicalDf = df.copy()
  showAtypicalValues(atypicalDf)

  writeDfToCsv(df, os.path.join(outputFolder, 'result.csv'))
  writeDfToCsv(df, os.path.join(otherOutputFolder, 'result.csv'))
  writeDfToCsv(df, os.path.join(otherOutputFolder2, 'result.csv'))

  return df

In [14]:
def visualizeData(mergedMainData):
  df = pd.DataFrame(mergedMainData)

  print(df.dtypes)
  print(df.count())

  columns = [columnName for columnName in df.columns]

  for column in columns:
    topDf = df[column].value_counts().head(dataVisualizationTopLimit)
    print(topDf)
    y_axis = list(reversed(topDf.index))
    x_axis = list(reversed(topDf.values))
    plt.ylabel(column)
    plt.barh(y_axis, x_axis)
    plt.show()

In [15]:
def main():
  # Definiendo el inicio del proceso
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))

  isPreprocessed = False

  # Leyendo la data obtenida en el entendimiento de los datos
  bumeranData = readCsvAsDf(os.path.join(inputMainFolder, 'result.csv')) # Ver si se puede cambiar a csv
  
  # Leyendo archivos maestros
  jobProfileName = readCsvAsDf(os.path.join(inputMastersFolder, 'nombrePerfilConvocatoria.csv'))
  lastWorkCompany = readCsvAsDf(os.path.join(inputMastersFolder, 'empresaUltimoTrabajo.csv'))
  lastWorkName = readCsvAsDf(os.path.join(inputMastersFolder, 'nombreUltimoTrabajo.csv'))
  lastEducationInstitution = readCsvAsDf(os.path.join(inputMastersFolder, 'institucionUltimoEstudio.csv'))
  lastEducationName = readCsvAsDf(os.path.join(inputMastersFolder, 'nombreUltimoEstudio.csv'))

  # Aplicando los datos de los maestros y validaciones
  preprocessedData = readCsvAsDict(os.path.join(outputFolder, 'result.csv')) if isPreprocessed else readAndWritePreprocessedData(bumeranData, [jobProfileName, lastWorkCompany, lastWorkName, lastEducationInstitution, lastEducationName])
  print("Se terminó el preprocesamiento")

  #visualizeData(preprocessedData)

  # Definiendo el fin del proceso
  endTime = datetime.datetime.now()
  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))

In [16]:
if __name__ == "__main__":
  main()

Inicio: 2023-06-14 22:15:35.860526


,columnName,columnType,percentNulls
1,otrasHabilidades,Numérico,22.85%
2,habilidadesTecnicas,Numérico,11.83%
3,idiomas,Numérico,7.45%
4,diasUltimoEstudio,Numérico,6.87%
5,diasUltimoTrabajo,Numérico,5.72%
6,empresaUltimoTrabajo,Categórico,5.45%
7,aniosExperiencia,Numérico,5.44%
8,areaUltimoTrabajo,Categórico,5.38%
9,nombreUltimoTrabajo,Categórico,5.38%
10,numeroTrabajos,Numérico,5.38%


,columnName,columnType,percentNulls
1,otrasHabilidades,Numérico,22.85%
2,habilidadesTecnicas,Numérico,11.83%
3,idiomas,Numérico,7.45%
4,diasUltimoEstudio,Numérico,6.87%
5,diasUltimoTrabajo,Numérico,5.72%
6,aniosExperiencia,Numérico,5.44%
7,numeroTrabajos,Numérico,5.38%
8,sueldoPretendido,Numérico,3.83%
9,aniosEstudio,Numérico,1.25%
10,numeroEstudios,Numérico,1.21%


,columnName,columnType,percentNulls
1,nombrePerfilConvocatoria,Categórico,0.00%
2,paisUltimoEstudio,Categórico,0.00%
3,otrasHabilidades,Numérico,0.00%
4,idiomas,Numérico,0.00%
5,habilidadesTecnicas,Numérico,0.00%
6,numeroEstudios,Numérico,0.00%
7,aniosEstudio,Numérico,0.00%
8,gradoUltimoEstudio,Categórico,0.00%
9,estadoUltimoEstudio,Categórico,0.00%
10,nombreUltimoEstudio,Categórico,0.00%


,columnName,columnType,percentAtypical
1,idiomas,Numérico,3.55%
2,habilidadesTecnicas,Numérico,2.66%
3,otrasHabilidades,Numérico,2.41%
4,diasUltimoTrabajo,Numérico,2.27%
5,numeroEstudios,Numérico,2.25%
6,aniosEstudio,Numérico,1.63%
7,aniosExperiencia,Numérico,0.80%
8,sueldoPretendido,Numérico,0.42%
9,numeroTrabajos,Numérico,0.34%
10,diasUltimoEstudio,Numérico,0.12%


,columnName,columnType,percentAtypical
1,nombrePerfilConvocatoria,Categórico,0.00%
2,paisUltimoEstudio,Categórico,0.00%
3,otrasHabilidades,Numérico,0.00%
4,idiomas,Numérico,0.00%
5,habilidadesTecnicas,Numérico,0.00%
6,numeroEstudios,Numérico,0.00%
7,aniosEstudio,Numérico,0.00%
8,gradoUltimoEstudio,Categórico,0.00%
9,estadoUltimoEstudio,Categórico,0.00%
10,nombreUltimoEstudio,Categórico,0.00%


Se terminó el preprocesamiento
Fin: 2023-06-14 22:15:44.735072
Tiempo: 0:00:08.874546
